In [2]:
import torch
import torchvision
import matplotlib.pyplot as plt
from pathlib import Path
from torchvision.transforms import ToTensor
from torchvision import transforms
from torch.utils.data import DataLoader
import os
from torch import nn
from tqdm.auto import tqdm
from torchvision import models
from torchinfo import summary

In [3]:
train_dir_path = Path("E:/CODING/projects/data/food_data/train")
test_dir_path =  Path("E:/CODING/projects/data/food_data/test")

trans = transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.ToTensor()
])
cpu_c = os.cpu_count()
    

In [4]:
def make_data(train_dir_path, test_dir_path, trans, batch_size, num_workers):

    train_data = torchvision.datasets.ImageFolder(root=train_dir_path, transform=trans)
    test_data = torchvision.datasets.ImageFolder(root=test_dir_path, transform=trans)

    train_dataloader = DataLoader(dataset=train_data,
    batch_size=batch_size,
    shuffle=True,
    pin_memory=True,
    num_workers=num_workers)
    test_dataloader = DataLoader(dataset=test_data,
    batch_size=batch_size,
    shuffle=True,
    pin_memory=True,
    num_workers=num_workers)
    classes_names = train_data.classes

    return train_dataloader, test_dataloader, classes_names

In [5]:
from torch.cuda import is_available
device = "cuda" if is_available() else "cpu"
device

'cuda'

In [6]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct/len(y_pred)) * 100
    return acc

In [7]:
def train_model(model, train_dataloader, test_dataloader, loss_fn, optimizer, accuracy_fn, epochs, device):
    train_epoch_count = []
    train_loss_values = []
    train_acc_values = []
    test_epoch_count = []
    test_loss_values = []
    test_acc_values = []
    for epoch in tqdm(range(epochs)):
        # Train Block
        ls_loss = 0
        ls_acc = 0
        for batch, (X, y) in enumerate(train_dataloader):
            X, y = X.to(device), y.to(device)
            #y = y.type(torch.long)
            model.train()
            logits = model(X).squeeze()
            preds = torch.round(torch.sigmoid(logits))
            #print(X.shape, y.shape, logits.shape)
            loss = loss_fn(logits, y.float())
            acc = accuracy_fn(y_true = y, y_pred = preds)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            model.eval()
            ls_loss += loss.item()
            ls_acc += acc
            if batch % 400 == 0:
                print(f"Epoch: {epoch+1} | Batch: {batch}...")
        mean_loss = ls_loss / len(train_dataloader)
        mean_acc = ls_acc / len(train_dataloader)
        train_loss_values.append(loss.item())
        train_acc_values.append(acc)
        train_epoch_count.append(epoch+1)
        print(f"Epoch: {epoch+1} | Loss: {mean_loss:.4f} | Acc: {mean_acc:.2f}%")


        # Test Block 
        ls_loss, ls_acc = 0, 0
        for batch, (X, y) in enumerate(test_dataloader):
            with torch.inference_mode():
                model.eval()
                X, y = X.to(device), y.to(device)
                #y = y.type(torch.long)
                mean_loss, mean_acc = 0, 0
                logits = model(X).squeeze()
                preds = torch.round(torch.sigmoid(logits))
                loss = loss_fn(logits, y.float())
                acc = accuracy_fn(y_true = y,y_pred = preds)
                ls_loss += loss.item()
                ls_acc += acc
                if batch % 300 == 0:
                    print(f"TEST :    Epoch: {epoch+1} | Batch: {batch}...")
        mean_loss = ls_loss / len(test_dataloader)
        mean_acc = ls_acc / len(test_dataloader)
        test_loss_values.append(loss.item())
        test_acc_values.append(acc)
        test_epoch_count.append(epoch+1)
        print(f"TEST :    Epoch: {epoch+1} | Loss: {mean_loss:.4f} | Acc: {mean_acc:.2f}%\n\n")

    return train_loss_values, train_acc_values, test_loss_values, test_acc_values, train_epoch_count, test_epoch_count

In [8]:
def plots(train_loss_values, train_acc_values, test_loss_values, test_acc_values, train_epoch_count, test_epoch_count):
    plt.subplot(2, 1, 1)
    plt.plot(train_epoch_count, train_acc_values, c='g', label='Train Acc')
    plt.plot(train_epoch_count, test_acc_values, c='r', label='Test ACC')
    plt.title('Acc')
    plt.legend()
    plt.subplot(2, 1, 2)
    plt.plot(train_epoch_count, train_loss_values, c='blue', label="Train Loss")
    plt.plot(train_epoch_count, test_loss_values, c='orange', label="Test Loss")
    plt.title('Loss')
    plt.legend();

In [9]:
from torchvision import models
weights = models.EfficientNet_B0_Weights.DEFAULT

In [10]:
auto_transform = weights.transforms()

In [11]:
auto_transform

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [71]:
AlexV0_transfer_efficientnet_b0 = models.efficientnet_b0(weights=weights).to(device)

In [72]:
train_dataloader, test_dataloader, classes_names = make_data(train_dir_path=train_dir_path, test_dir_path=test_dir_path, trans=auto_transform, 
                                                            batch_size=32, num_workers=cpu_c)

In [73]:
AlexV0_transfer_efficientnet_b0

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [24]:
! pip install torchinfo

In [74]:
from torchinfo import summary
summary(model=AlexV0_transfer_efficientnet_b0,
        input_size=(32, 3, 224, 224),
        col_names=['input_size', 'output_size',
                   "num_params", 'trainable'],
                   col_width=20,
                   row_settings=['var_names'])


Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 1000]           --                   True
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   True
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   True
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   864                  True
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   64                   True
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 16, 112

In [26]:
for params in AlexV0_transfer_efficientnet_b0.features.parameters():
   params.requires_grad=False
from torchinfo import summary
summary(model=AlexV0_transfer_efficientnet_b0,
        input_size=(32, 3, 224, 224),
        col_names=['input_size', 'output_size',
                   "num_params", 'trainable'],
                   col_width=20,
                   row_settings=['var_names'])


Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 1000]           --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 

In [75]:
AlexV0_transfer_efficientnet_b0.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True),
    torch.nn.Linear(in_features=1280, out_features=1, bias=True)
).to(device)

In [ ]:
optimizerV0 = torch.optim.SGD(AlexV0_transfer_efficientnet_b0.parameters(),
                              lr=0.01)
loss_fn = nn.BCEWithLogitsLoss()
train_loss_valuesV0, train_acc_valuesV0, test_loss_valuesV0, test_acc_valuesV0, train_epoch_countV0, test_epoch_countV0 = train_model(
    model=AlexV0_transfer_efficientnet_b0, train_dataloader=train_dataloader, test_dataloader=test_dataloader, 
            optimizer=optimizerV0, loss_fn=loss_fn ,accuracy_fn=accuracy_fn, epochs=10, device=device)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | Batch: 0...
Epoch: 1 | Batch: 400...
Epoch: 1 | Batch: 800...
Epoch: 1 | Batch: 1200...
Epoch: 1 | Loss: 0.2871 | Acc: 88.34%
TEST :    Epoch: 1 | Batch: 0...
TEST :    Epoch: 1 | Batch: 300...


 10%|█         | 1/10 [06:05<54:50, 365.60s/it]

TEST :    Epoch: 1 | Loss: 0.2085 | Acc: 92.35%


Epoch: 2 | Batch: 0...
Epoch: 2 | Batch: 400...
Epoch: 2 | Batch: 800...
Epoch: 2 | Batch: 1200...
Epoch: 2 | Loss: 0.1859 | Acc: 92.80%
TEST :    Epoch: 2 | Batch: 0...
TEST :    Epoch: 2 | Batch: 300...


 20%|██        | 2/10 [11:44<46:40, 350.03s/it]

...

TEST :    Epoch: 8 | Loss: 0.1812 | Acc: 93.76%


Epoch: 9 | Batch: 0...
Epoch: 9 | Batch: 400...
Epoch: 9 | Batch: 800...
Epoch: 9 | Batch: 1200...
Epoch: 9 | Loss: 0.0845 | Acc: 96.80%
TEST :    Epoch: 9 | Batch: 0...
TEST :    Epoch: 9 | Batch: 300...


 90%|█████████ | 9/10 [52:44<05:57, 357.76s/it]

TEST :    Epoch: 9 | Loss: 0.2143 | Acc: 93.05%


Epoch: 10 | Batch: 0...
Epoch: 10 | Batch: 400...
Epoch: 10 | Batch: 800...
Epoch: 10 | Batch: 1200...
Epoch: 10 | Loss: 0.0776 | Acc: 97.06%
TEST :    Epoch: 10 | Batch: 0...
TEST :    Epoch: 10 | Batch: 300...


100%|██████████| 10/10 [58:46<00:00, 352.60s/it]

TEST :    Epoch: 10 | Loss: 0.1982 | Acc: 94.58%




In [9]:
def save_model(model, MODEL_PATH, model_name):
    SAVE_PATH = MODEL_PATH / model_name
    torch.save(obj = model.state_dict(),
               f = SAVE_PATH)
    print(f"Saved to: {SAVE_PATH}")

In [46]:
MODEL_PATH_AlexV0_transfer_efficientnet_b0 = Path("E:/CODING/projects/transfer_learning_models")
MODEL_NAME_AlexV0_transfer_efficientnet_b0 = "AlexV0_transfer_efficientnet_b0.pth"
MODEL_PATH_AlexV0_transfer_efficientnet_b0 / MODEL_NAME_AlexV0_transfer_efficientnet_b0
save_model(AlexV0_transfer_efficientnet_b0, MODEL_PATH_AlexV0_transfer_efficientnet_b0, MODEL_NAME_AlexV0_transfer_efficientnet_b0)

Saved to: E:\CODING\projects\transfer_learning_models\AlexV0_transfer_efficientnet_b0.pth


In [8]:
def made_trans_models(tl_weights, tl_model, device):
    weights = getattr(models, tl_weights).DEFAULT
    auto_transform = weights.transforms()
    name_model = getattr(models, tl_model)(weights = weights).to(device)
 #print(summary(model=name_model,input_size=(32, 3, 224, 224),col_names=['input_size', 'output_size',"num_params", 'trainable'],col_width=20,row_settings=['var_names']))
    print("Off requires_grad...\n")
    for params in name_model.features.parameters():
        params.requires_grad=False
    #print(summary(model=name_model,input_size=(32, 3, 224, 224),col_names=['input_size', 'output_size',"num_params", 'trainable'],col_width=20,row_settings=['var_names']))
    print(name_model)
    return name_model, auto_transform

In [31]:
device

'cuda'

In [33]:
AlexV0_transfer_Alexnet, auto_transform = made_trans_models(tl_weights='AlexNet_Weights', tl_model='alexnet', device=device)

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to C:\Users\aleks/.cache\torch\hub\checkpoints\alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:22<00:00, 10.7MB/s] 


Off requires_grad...

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_featu

In [35]:
AlexV0_transfer_Alexnet.to(device)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [36]:
next(AlexV0_transfer_Alexnet.parameters()).is_cuda

True

In [37]:
for params in AlexV0_transfer_Alexnet.features.parameters():
    params.requires_grad = False

In [38]:
AlexV0_transfer_Alexnet.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.5, inplace=False),
    torch.nn.Linear(in_features=9216, out_features=4096, bias=True),
    torch.nn.ReLU(inplace=True),
    torch.nn.Dropout(p=0.5, inplace=False),
    torch.nn.Linear(in_features=4096, out_features=4096, bias=True),
    torch.nn.ReLU(inplace=True),
    torch.nn.Linear(in_features=4096, out_features=1, bias=True)
  ).to(device)


In [39]:
train_dataloader, test_dataloader, classes_names = make_data(
    train_dir_path=train_dir_path, test_dir_path=test_dir_path, trans=auto_transform, 
                                                            batch_size=32, num_workers=cpu_c)

In [40]:
optimizerV1 = torch.optim.SGD(AlexV0_transfer_Alexnet.parameters(),
                              lr=0.01)
loss_fn = nn.BCEWithLogitsLoss()
train_loss_valuesV1, train_acc_valuesV1, test_loss_valuesV1, test_acc_valuesV1, train_epoch_countV1, test_epoch_countV1 = train_model(
    model=AlexV0_transfer_Alexnet, train_dataloader=train_dataloader, test_dataloader=test_dataloader, 
            optimizer=optimizerV1, loss_fn=loss_fn ,accuracy_fn=accuracy_fn, epochs=10, device=device)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | Batch: 0...
Epoch: 1 | Batch: 400...
Epoch: 1 | Batch: 800...
Epoch: 1 | Batch: 1200...
Epoch: 1 | Loss: 0.1490 | Acc: 94.39%
TEST :    Epoch: 1 | Batch: 0...
TEST :    Epoch: 1 | Batch: 300...


 10%|█         | 1/10 [03:46<33:56, 226.23s/it]

TEST :    Epoch: 1 | Loss: 0.1546 | Acc: 94.30%


Epoch: 2 | Batch: 0...
Epoch: 2 | Batch: 400...
Epoch: 2 | Batch: 800...
Epoch: 2 | Batch: 1200...
Epoch: 2 | Loss: 0.1012 | Acc: 96.14%
TEST :    Epoch: 2 | Batch: 0...
TEST :    Epoch: 2 | Batch: 300...


 20%|██        | 2/10 [06:31<25:25, 190.66s/it]

...

TEST :    Epoch: 8 | Loss: 0.1592 | Acc: 94.40%


Epoch: 9 | Batch: 0...
Epoch: 9 | Batch: 400...
Epoch: 9 | Batch: 800...
Epoch: 9 | Batch: 1200...
Epoch: 9 | Loss: 0.0276 | Acc: 98.98%
TEST :    Epoch: 9 | Batch: 0...
TEST :    Epoch: 9 | Batch: 300...


 90%|█████████ | 9/10 [24:22<02:32, 152.75s/it]

TEST :    Epoch: 9 | Loss: 0.1359 | Acc: 95.34%


Epoch: 10 | Batch: 0...
Epoch: 10 | Batch: 400...
Epoch: 10 | Batch: 800...
Epoch: 10 | Batch: 1200...
Epoch: 10 | Loss: 0.0232 | Acc: 99.11%
TEST :    Epoch: 10 | Batch: 0...
TEST :    Epoch: 10 | Batch: 300...


100%|██████████| 10/10 [26:49<00:00, 160.99s/it]

TEST :    Epoch: 10 | Loss: 0.1749 | Acc: 94.21%




In [47]:
MODEL_PATH_AlexV0_transfer_Alexnet = Path("E:/CODING/projects/transfer_learning_models")
MODEL_NAME_AlexV0_transfer_Alexnet = "AlexV0_transfer_Alexnet.pth"
MODEL_PATH_AlexV0_transfer_Alexnet / MODEL_NAME_AlexV0_transfer_Alexnet
save_model(AlexV0_transfer_Alexnet, MODEL_PATH_AlexV0_transfer_Alexnet, MODEL_NAME_AlexV0_transfer_Alexnet)

Saved to: E:\CODING\projects\transfer_learning_models\AlexV0_transfer_Alexnet.pth


In [41]:
print(summary(model=AlexV0_transfer_Alexnet,input_size=(32, 3, 224, 224),col_names=['input_size', 'output_size',"num_params", 'trainable'],col_width=20,row_settings=['var_names']))

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
AlexNet (AlexNet)                        [32, 3, 224, 224]    [32, 1]              --                   Partial
├─Sequential (features)                  [32, 3, 224, 224]    [32, 256, 6, 6]      --                   False
│    └─Conv2d (0)                        [32, 3, 224, 224]    [32, 64, 55, 55]     (23,296)             False
│    └─ReLU (1)                          [32, 64, 55, 55]     [32, 64, 55, 55]     --                   --
│    └─MaxPool2d (2)                     [32, 64, 55, 55]     [32, 64, 27, 27]     --                   --
│    └─Conv2d (3)                        [32, 64, 27, 27]     [32, 192, 27, 27]    (307,392)            False
│    └─ReLU (4)                          [32, 192, 27, 27]    [32, 192, 27, 27]    --                   --
│    └─MaxPool2d (5)                     [32, 192, 27, 27]    [32, 192, 13, 13]    --                   --
│    └─Conv2d (6

In [ ]:
ConvNeXt_weights = models.convnext_tiny

In [ ]:
AlexV0_ConvNeXt_tiny, auto_transform = made_trans_models(tl_weights='ConvNeXt_Tiny_Weights', tl_model='convnext_tiny', device=device)

Downloading: "https://download.pytorch.org/models/convnext_tiny-983f1562.pth" to C:\Users\aleks/.cache\torch\hub\checkpoints\convnext_tiny-983f1562.pth
100%|██████████| 109M/109M [00:10<00:00, 10.7MB/s] 


Off requires_grad...

ConvNeXt(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm2d((96,), eps=1e-06, elementwise_affine=True)
    )
    (1): Sequential(
      (0): CNBlock(
        (block): Sequential(
          (0): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (1): Permute()
          (2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_features=96, out_features=384, bias=True)
          (4): GELU(approximate='none')
          (5): Linear(in_features=384, out_features=96, bias=True)
          (6): Permute()
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): CNBlock(
        (block): Sequential(
          (0): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (1): Permute()
          (2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (3

In [55]:
AlexV0_ConvNeXt_tiny.classifier = nn.Sequential(
    nn.LayerNorm((768,), eps=1e-06, elementwise_affine=True),
    nn.Flatten(start_dim=1, end_dim=-1),
    nn.Linear(in_features=768, out_features=1, bias=True)
).to(device)

ImageClassification(
    crop_size=[224]
    resize_size=[236]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [56]:
next(AlexV0_ConvNeXt_tiny.parameters()).is_cuda

True

In [58]:
train_dataloader, test_dataloader, classes_names = make_data(
    train_dir_path=train_dir_path, test_dir_path=test_dir_path, trans=auto_transform, 
                                                            batch_size=32, num_workers=cpu_c)

In [59]:
optimizerV2 = torch.optim.SGD(AlexV0_ConvNeXt_tiny.parameters(),
                              lr=0.01)
loss_fn = nn.BCEWithLogitsLoss()
train_loss_valuesV2, train_acc_valuesV2, test_loss_valuesV2, test_acc_valuesV2, train_epoch_countV2, test_epoch_countV2 = train_model(
    model=AlexV0_ConvNeXt_tiny, train_dataloader=train_dataloader, test_dataloader=test_dataloader, 
            optimizer=optimizerV2, loss_fn=loss_fn ,accuracy_fn=accuracy_fn, epochs=10, device=device)

  0%|          | 0/10 [00:41<?, ?it/s]


RuntimeError: Given normalized_shape=[768], expected input with shape [*, 768], but got input of size[32, 768, 1, 1]

In [ ]:
cong = models.efficientnet_b1

In [12]:
AlexV3_efficientnetb1, auto_transform = made_trans_models(tl_weights='EfficientNet_B1_Weights', tl_model='efficientnet_b1', device=device)

Downloading: "https://download.pytorch.org/models/efficientnet_b1-c27df63c.pth" to C:\Users\aleks/.cache\torch\hub\checkpoints\efficientnet_b1-c27df63c.pth
100%|██████████| 30.1M/30.1M [00:02<00:00, 10.8MB/s]


Off requires_grad...

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          

In [13]:
auto_transform

ImageClassification(
    crop_size=[240]
    resize_size=[255]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [17]:
print(summary(model=AlexV3_efficientnetb1,input_size=(32, 3, 255, 255),col_names=['input_size', 'output_size',"num_params", 'trainable'],col_width=20,row_settings=['var_names']))

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 255, 255]    [32, 1000]           --                   Partial
├─Sequential (features)                                      [32, 3, 255, 255]    [32, 1280, 8, 8]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 255, 255]    [32, 32, 128, 128]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 255, 255]    [32, 32, 128, 128]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 128, 128]   [32, 32, 128, 128]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 128, 128]   [32, 32, 128, 128]   --                   --
│    └─Sequential (1)                                        [32, 32, 128, 128]   [32, 

In [18]:
AlexV3_efficientnetb1.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, out_features=1, bias=True)
  ).to(device)


In [20]:
next(AlexV3_efficientnetb1.parameters()).is_cuda

True

In [21]:
train_dataloader, test_dataloader, classes_names = make_data(train_dir_path=train_dir_path, test_dir_path=test_dir_path, trans=auto_transform, 
                                                            batch_size=32, num_workers=cpu_c)

In [22]:
optimizerV3 = torch.optim.SGD(AlexV3_efficientnetb1.parameters(),
                              lr=0.01)
loss_fn = nn.BCEWithLogitsLoss()
train_loss_valuesV3, train_acc_valuesV3, test_loss_valuesV3, test_acc_valuesV3, train_epoch_countV3, test_epoch_countV3 = train_model(
    model=AlexV3_efficientnetb1, train_dataloader=train_dataloader, test_dataloader=test_dataloader, 
            optimizer=optimizerV3, loss_fn=loss_fn ,accuracy_fn=accuracy_fn, epochs=10, device=device)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | Batch: 0...
Epoch: 1 | Batch: 400...
Epoch: 1 | Batch: 800...
Epoch: 1 | Batch: 1200...
Epoch: 1 | Loss: 0.2923 | Acc: 94.69%
TEST :    Epoch: 1 | Batch: 0...
TEST :    Epoch: 1 | Batch: 300...


 10%|█         | 1/10 [03:56<35:32, 236.92s/it]

TEST :    Epoch: 1 | Loss: 0.1953 | Acc: 94.66%


Epoch: 2 | Batch: 0...
Epoch: 2 | Batch: 400...
Epoch: 2 | Batch: 800...
Epoch: 2 | Batch: 1200...
Epoch: 2 | Loss: 0.1579 | Acc: 96.20%
TEST :    Epoch: 2 | Batch: 0...
TEST :    Epoch: 2 | Batch: 300...


 20%|██        | 2/10 [10:44<45:00, 337.58s/it]

...

TEST :    Epoch: 8 | Loss: 0.1053 | Acc: 96.29%


Epoch: 9 | Batch: 0...
Epoch: 9 | Batch: 400...
Epoch: 9 | Batch: 800...
Epoch: 9 | Batch: 1200...
Epoch: 9 | Loss: 0.0940 | Acc: 96.97%
TEST :    Epoch: 9 | Batch: 0...
TEST :    Epoch: 9 | Batch: 300...


 90%|█████████ | 9/10 [1:20:17<09:41, 581.59s/it]

TEST :    Epoch: 9 | Loss: 0.0947 | Acc: 96.74%


Epoch: 10 | Batch: 0...
Epoch: 10 | Batch: 400...
Epoch: 10 | Batch: 800...
Epoch: 10 | Batch: 1200...
Epoch: 10 | Loss: 0.0913 | Acc: 97.06%
TEST :    Epoch: 10 | Batch: 0...
TEST :    Epoch: 10 | Batch: 300...


100%|██████████| 10/10 [1:30:44<00:00, 544.47s/it]

TEST :    Epoch: 10 | Loss: 0.0938 | Acc: 96.75%




In [ ]:
asd = models.EfficientNet_V2_S_Weights

In [23]:
AlexV3_efficientnet_v2_s, auto_transform = made_trans_models(tl_weights='EfficientNet_V2_S_Weights', tl_model='efficientnet_v2_s', device=device)

Downloading: "https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth" to C:\Users\aleks/.cache\torch\hub\checkpoints\efficientnet_v2_s-dd5fe13b.pth
100%|██████████| 82.7M/82.7M [00:07<00:00, 11.0MB/s]


Off requires_grad...

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_

In [24]:
auto_transform

ImageClassification(
    crop_size=[384]
    resize_size=[384]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [25]:
print(summary(model=AlexV3_efficientnet_v2_s,input_size=(32, 3, 384, 384),col_names=['input_size', 'output_size',"num_params", 'trainable'],col_width=20,row_settings=['var_names']))

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 384, 384]    [32, 1000]           --                   Partial
├─Sequential (features)                                      [32, 3, 384, 384]    [32, 1280, 12, 12]   --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 384, 384]    [32, 24, 192, 192]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 384, 384]    [32, 24, 192, 192]   (648)                False
│    │    └─BatchNorm2d (1)                                  [32, 24, 192, 192]   [32, 24, 192, 192]   (48)                 False
│    │    └─SiLU (2)                                         [32, 24, 192, 192]   [32, 24, 192, 192]   --                   --
│    └─Sequential (1)                                        [32, 24, 192, 192]   [32, 

In [28]:
AlexV3_efficientnet_v2_s.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, out_features=1, bias=True)
).to(device)
next(AlexV3_efficientnet_v2_s.parameters()).device

device(type='cuda', index=0)

In [29]:
train_dataloader, test_dataloader, classes_names = make_data(train_dir_path=train_dir_path, test_dir_path=test_dir_path, trans=auto_transform, 
                                                            batch_size=32, num_workers=cpu_c)
optimizerV4 = torch.optim.SGD(AlexV3_efficientnet_v2_s.parameters(),
                              lr=0.01)
loss_fn = nn.BCEWithLogitsLoss()
train_loss_valuesV4, train_acc_valuesV4, test_loss_valuesV4, test_acc_valuesV4, train_epoch_countV4, test_epoch_countV4 = train_model(
    model=AlexV3_efficientnet_v2_s, train_dataloader=train_dataloader, test_dataloader=test_dataloader, 
            optimizer=optimizerV4, loss_fn=loss_fn ,accuracy_fn=accuracy_fn, epochs=10, device=device)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | Batch: 0...
Epoch: 1 | Batch: 400...
Epoch: 1 | Batch: 800...
Epoch: 1 | Batch: 1200...
Epoch: 1 | Loss: 0.1706 | Acc: 95.83%
TEST :    Epoch: 1 | Batch: 0...
TEST :    Epoch: 1 | Batch: 300...


 10%|█         | 1/10 [11:10<1:40:33, 670.44s/it]

TEST :    Epoch: 1 | Loss: 0.1073 | Acc: 96.77%


Epoch: 2 | Batch: 0...
Epoch: 2 | Batch: 400...
Epoch: 2 | Batch: 800...
Epoch: 2 | Batch: 1200...
Epoch: 2 | Loss: 0.1001 | Acc: 96.87%
TEST :    Epoch: 2 | Batch: 0...
TEST :    Epoch: 2 | Batch: 300...


 20%|██        | 2/10 [20:52<1:22:27, 618.50s/it]

...

TEST :    Epoch: 8 | Loss: 0.0782 | Acc: 97.09%


Epoch: 9 | Batch: 0...
Epoch: 9 | Batch: 400...
Epoch: 9 | Batch: 800...
Epoch: 9 | Batch: 1200...
Epoch: 9 | Loss: 0.0677 | Acc: 97.58%
TEST :    Epoch: 9 | Batch: 0...
TEST :    Epoch: 9 | Batch: 300...


 90%|█████████ | 9/10 [3:03:58<32:17, 1937.95s/it]

TEST :    Epoch: 9 | Loss: 0.0664 | Acc: 97.60%


Epoch: 10 | Batch: 0...
Epoch: 10 | Batch: 400...
Epoch: 10 | Batch: 800...
Epoch: 10 | Batch: 1200...
Epoch: 10 | Loss: 0.0664 | Acc: 97.64%
TEST :    Epoch: 10 | Batch: 0...
TEST :    Epoch: 10 | Batch: 300...


100%|██████████| 10/10 [3:39:12<00:00, 1315.26s/it]

TEST :    Epoch: 10 | Loss: 0.0721 | Acc: 97.28%




In [ ]:
MODEL_PATH_AlexV3_efficient_v2_s = Path("E:/CODING/projects/transfer_learning_models")
MODEL_NAME_AlexV3_efficient_v2_s = "AlexV3_efficient_v2_s.pth"
MODEL_PATH_AlexV3_efficient_v2_s / MODEL_NAME_AlexV3_efficient_v2_s
save_model(AlexV3_efficientnet_v2_s, MODEL_PATH_AlexV3_efficient_v2_s, MODEL_NAME_AlexV3_efficient_v2_s)

Saved to: E:\CODING\projects\transfer_learning_models\AlexV3_efficient_v2_s.pth


In [32]:
MODEL_PATH_AlexV3_efficientnetb1 = Path("E:/CODING/projects/transfer_learning_models")
MODEL_NAME_AlexV3_efficientnetb1 = "AlexV3_efficientnetb1.pth"
MODEL_PATH_AlexV3_efficientnetb1 / MODEL_NAME_AlexV3_efficientnetb1
save_model(AlexV3_efficientnetb1, MODEL_PATH_AlexV3_efficientnetb1, MODEL_NAME_AlexV3_efficientnetb1)

Saved to: E:\CODING\projects\transfer_learning_models\AlexV3_efficientnetb1.pth
